In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import preprocessing as preprocessing
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np

In [ ]:
'''train_df = pd.read_csv('../../datasets/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv')
test_df = pd.read_csv('../../datasets/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv')'''

train_df = pd.read_csv('a. IDRiD_Disease Grading_Training Labels.csv')
test_df = pd.read_csv('b. IDRiD_Disease Grading_Testing Labels.csv')
print(train_df.shape)
print(test_df.shape)
train_df.head()
print(train_df.shape)
print(test_df.shape)
train_df.head()

In [ ]:
def process(i):
    img = cv2.imread(i)
    img = img[0:3000,300:3670]
    width = 299
    height = 299
    dim = (width, height)

    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    blurred = cv2.blur(resized, ksize=(15, 15))
    diff = resized - blurred
    return diff

In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 299, 299, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(train_df['Image name'])):
    #print(image_id)
    #print(f'../../datasets/B. Disease Grading/1. Original Images/a. Training Set/{image_id}.jpg')
    x_train[i, :, :, :] = process(
        f'drive/My Drive/1. Original Images/a. Training Set/{image_id}.jpg'
    )

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 299, 299, 3), dtype=np.uint8)
for i, image_id in enumerate(tqdm(test_df['Image name'])):
    x_train[i, :, :, :] = process(
        f'drive/My Drive/1. Original Images/b. Testing Set/{image_id}.jpg'
    )

In [ ]:
y_train = (train_df['Retinopathy grade']).values
y_test = (test_df['Retinopathy grade']).values

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

plt.imshow(x_train[0])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, 
    test_size=0.1, 
    random_state=2020
)

In [ ]:
from keras.utils import to_categorical
print(y_train.shape)
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print(y_train.shape)
print(y_val.shape)


In [ ]:
img_width, img_height = 299, 299
nb_train_samples = 413
nb_validation_samples = 103
epochs = 300

model = applications.InceptionV3(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [ ]:
batch_size = 8
# Freeze the layers 
for layer in model.layers:
    layer.trainable = False

In [ ]:
#Adding custom Layers 
x = model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "cosine_proximity", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

model_final.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=0, 
                                   # zoom_range = 0.3, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=batch_size, seed=10)
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_val, y_val, shuffle=False, batch_size=batch_size, seed=10)

In [ ]:
import keras as keras
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inception_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch=x_train.shape[0] / batch_size,
epochs=epochs,
validation_data = val_generator,
callbacks = [checkpoint, early,plot_losses]
verbose = 2)

In [ ]:
model.load_weights('inception_1.h5')
y_test = model.predict(x_test, verbose=2)
y_pred = y_test.argmax(axis=1)
print(y_pred.shape)


In [ ]:
lb = LabelBinarizer()
    lb.fit(y_test)
y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
print(roc_auc_score(y_test, y_pred, average="macro"))

In [ ]:
print accuracy_score(y_test, y_pred)